In [2]:
import pandas as pd
import os

best_files = [
    #'../Submissions/submission_auto_fine_tune_combined_0117.csv',
    'submission_09_soft_pl_n_valid_resnet_origin_oof_augment_minmax.csv', #1331
    #'submission_dpn_lee_out_of_fold_avg_with_augmentation_40_avg.csv', #1598
    #'submission_resnet_polar_cpl_oof_augment_minmax_median.csv',
    #'submission_nn_resnet_polar_oof_augment_minmax_median.csv', #0.1717
    #'submission_nn_resnet_polar_soft_pl_n_valid_oof_augment_avg.csv',
    #'../Submissions/submission_auto_fine_tune_combined_0117.csv',
    #'submission_ensemble_resnet_oof_augment_minmax_median.csv', # 0.183 for avg
    #'submission_sswae_oof_augment_minmax_median.csv', #.1982 for avg
    
    'others/subm_blend009_2017-10-31-07-26.csv', # least correlated
    #'others/sub.csv',# second least correlated,
    #'others/submission.csv',
    'others/200_ens_densenet.csv', #1530
    'others/ens_ice_98989898989898989.csv', #1520
    #'others/submission_1low_test.csv', #?
    'others/submission_1516_test.csv', #1516 (good, and low corr)
    'others/LB1541_final_ensemble.csv', #1541 public lb (good, and low corr)
    'others/vggbnw_fcn_en_ordered.csv',
    '../Submissions/submission_ensemble_contrast_resnet_top3_oof_augment_avg.csv'
    #'others/',
    #'others/',
]

blen_col = 'is_iceberg_1'
file_num = len(best_files)

In [3]:
outs = [pd.read_csv(f, index_col=0) for f in best_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head(5)

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7
0,5941774d,0.006182,0.094676,1.943216e-02,0.022661,0.015196,0.006448,0.030369,0.006990
1,4023181e,0.249994,0.952222,3.168809e-02,0.281293,0.939407,0.758083,0.364333,0.000000
2,b20200e4,0.000003,0.167771,4.000000e-08,0.065910,0.040264,0.041054,0.017298,0.000000
3,e7f018bb,0.999321,0.989356,9.925741e-01,0.995712,0.999260,0.996372,0.998199,0.976659
4,4371c8c3,0.001795,0.900321,2.215107e-02,0.042113,0.828124,0.007940,0.657598,0.000000


In [4]:
for f in best_files:
    print(f)
concat_sub.corr()

submission_09_soft_pl_n_valid_resnet_origin_oof_augment_minmax.csv
others/subm_blend009_2017-10-31-07-26.csv
others/200_ens_densenet.csv
others/ens_ice_98989898989898989.csv
others/submission_1516_test.csv
others/LB1541_final_ensemble.csv
others/vggbnw_fcn_en_ordered.csv
../Submissions/submission_ensemble_contrast_resnet_top3_oof_augment_avg.csv


,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7
is_iceberg_0,1.000000,0.455086,0.947025,0.930582,0.660642,0.685990,0.857050,0.887997
is_iceberg_1,0.455086,1.000000,0.420791,0.555389,0.518634,0.689161,0.563222,0.343071
is_iceberg_2,0.947025,0.420791,1.000000,0.951330,0.654709,0.659503,0.847724,0.935989
is_iceberg_3,0.930582,0.555389,0.951330,1.000000,0.726105,0.785668,0.884432,0.885042
is_iceberg_4,0.660642,0.518634,0.654709,0.726105,1.000000,0.719479,0.741382,0.611892
is_iceberg_5,0.685990,0.689161,0.659503,0.785668,0.719479,1.000000,0.750361,0.582134
is_iceberg_6,0.857050,0.563222,0.847724,0.884432,0.741382,0.750361,1.000000,0.803360
is_iceberg_7,0.887997,0.343071,0.935989,0.885042,0.611892,0.582134,0.803360,1.000000


In [5]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.67
cutoff_hi = 0.33

In [6]:
import numpy as np
from scipy.stats import trim_mean, gmean

# get the data fields ready for stacking
print('Using columns: ', concat_sub.iloc[:, 1:file_num+1].columns.tolist())
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:file_num+1].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:file_num+1].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:file_num+1].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:file_num+1].median(axis=1)
concat_sub['25_percentile'] = np.percentile(concat_sub.iloc[:, 1:file_num+1], 50, axis=1, keepdims=True)
concat_sub['75_percentile'] = np.percentile(concat_sub.iloc[:, 1:file_num+1], 75, axis=1, keepdims=True)
concat_sub['trim_mean_01'] = trim_mean(concat_sub.iloc[:, 2:file_num+1], 0.1, axis=1) # except best file
concat_sub['trim_mean_02'] = trim_mean(concat_sub.iloc[:, 2:file_num+1], 0.2, axis=1) # except best file
concat_sub['is_iceberg_gmean'] = gmean(concat_sub.iloc[:, 1:file_num+1], axis=1)

Using columns:  ['is_iceberg_0', 'is_iceberg_1', 'is_iceberg_2', 'is_iceberg_3', 'is_iceberg_4', 'is_iceberg_5', 'is_iceberg_6', 'is_iceberg_7']


C:\Users\khyeh\Anaconda3\lib\site-packages\scipy\stats\stats.py:305: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


In [7]:
print(concat_sub.head(10))

         id  is_iceberg_0  is_iceberg_1  is_iceberg_2  is_iceberg_3  \
0  5941774d      0.006182      0.094676  1.943216e-02      0.022661   
1  4023181e      0.249994      0.952222  3.168809e-02      0.281293   
2  b20200e4      0.000003      0.167771  4.000000e-08      0.065910   
3  e7f018bb      0.999321      0.989356  9.925741e-01      0.995712   
4  4371c8c3      0.001795      0.900321  2.215107e-02      0.042113   
5  a8d9b1fd      0.013521      0.422845  5.195905e-02      0.292252   
6  29e7727e      0.023397      0.004942  3.701379e-01      0.277460   
7  92a51ffb      0.999117      0.997785  9.962149e-01      0.998744   
8  c769ac97      0.001959      0.002571  2.664300e-04      0.000038   
9  aee0547d      0.000753      0.001068  2.000000e-08      0.000002   

   is_iceberg_4  is_iceberg_5  is_iceberg_6  is_iceberg_7  is_iceberg_max  \
0      0.015196      0.006448      0.030369      0.006990        0.094676   
1      0.939407      0.758083      0.364333      0.000000       

In [7]:
#blen_col

concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:file_num+1] >= cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], # all > lo
                                    np.where(np.all(concat_sub.iloc[:,1:file_num+1] <= cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], # all < hi 
                                             np.where(concat_sub[blen_col] > .85,
                                                      concat_sub[blen_col],
                                                      concat_sub['is_iceberg_median'] # use the best
                                                     )
                                            )
                                   )


'''
                                             np.where(concat_sub['is_iceberg_0'] > .5,
                                                      concat_sub['is_iceberg_mean'],
                                                      concat_sub['is_iceberg_gmean'],
                                                     )
'''
#concat_sub['is_iceberg'] = .4 * concat_sub['is_iceberg_0'] + .6 * concat_sub['trim_mean_02']
concat_sub[['id', 'is_iceberg']].to_csv('blend_minimax_median_blend_corrected.csv', 
                                        index=False, float_format='%.15f')

In [9]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:file_num+1] >= cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], # all > lo
                                    np.where(np.all(concat_sub.iloc[:,1:file_num+1] <= cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], # all < hi 
                                             concat_sub['is_iceberg_median'] # use the best
                                            )
                                   )


'''
                                             np.where(concat_sub['is_iceberg_0'] > .5,
                                                      concat_sub['is_iceberg_mean'],
                                                      concat_sub['is_iceberg_gmean'],
                                                     )
'''
#concat_sub['is_iceberg'] = .4 * concat_sub['is_iceberg_0'] + .6 * concat_sub['trim_mean_02']
concat_sub[['id', 'is_iceberg']].to_csv('blend_minimax_median.csv', 
                                        index=False, float_format='%.15f')

In [10]:
test = pd.read_json('../Data/test.json', dtype={'inc_angle': str})
read_ids = pd.read_csv('real ids.csv')['id'].values
print(len(read_ids))

3425


In [11]:
th=6
leak_df = pd.read_csv('data_leakage_th_{}.csv'.format(th))
leak_angles = leak_df.inc_angle.values
leak_ice = leak_df.is_iceberg.values
print('max change size: ', len(leak_ice))


answers = concat_sub['is_iceberg'].values

change_counter = 0
wrong_ids = []

for i in range(test.shape[0]):
    if test.id.values[i] in read_ids and test.inc_angle.values[i] in leak_angles:
        
        ix, = np.where( leak_angles==test.inc_angle.values[i] )
        new_ans = leak_ice[ix]
        new_ans = max(0.0001, min(new_ans, 0.9999))
        
        print(test.id.values[i], end=': ')
        print('change from {:.5f} to {:.5f}'.format(answers[i], new_ans))
        
        if (new_ans < .5 and answers[i] >= .5) or (new_ans >= .5 and answers[i] < .5):
            change_counter += 1
            wrong_ids.append(test.id.values[i])
            
        answers[i] = new_ans
            
print('Final Changed count (contradict ones): ', change_counter)
print(wrong_ids)
concat_sub['is_iceberg'] = answers

max change size:  39
5941774d: change from 0.00618 to 0.00010
aee0547d: change from 0.00000 to 0.00010
565b28ac: change from 0.00000 to 0.00010
5f6d3988: change from 0.99142 to 0.99990
f5b4e3df: change from 0.97243 to 0.99990
6d868590: change from 0.99344 to 0.99990
a7bfd784: change from 0.99473 to 0.99990
89dbe7f7: change from 0.99925 to 0.99990
1ac8bb92: change from 0.99857 to 0.99990
956adc4d: change from 0.99941 to 0.99990
b130a0b7: change from 0.99982 to 0.99990
3d7e09d8: change from 0.98865 to 0.99990
d8c2bfe2: change from 0.99669 to 0.99990
50984a87: change from 0.78120 to 0.99990
fa94957a: change from 0.99803 to 0.99990
150e23c4: change from 0.99948 to 0.99990
078c6d99: change from 0.98937 to 0.99990
5ac74969: change from 0.98733 to 0.99990
9efcc11f: change from 0.00000 to 0.00010
fb611563: change from 0.99989 to 0.99990
0864e334: change from 0.97419 to 0.99990
1b371d8b: change from 0.93071 to 0.99990
4b82e7f7: change from 0.99988 to 0.99990
41b02e58: change from 0.99881 to 0.9

ba529f7f: change from 0.96340 to 0.99990
00798379: change from 0.97803 to 0.99990
41cc3cf1: change from 0.77459 to 0.99990
75d15e82: change from 0.99894 to 0.99990
3ea7f47e: change from 0.99799 to 0.99990
23e4de19: change from 0.99983 to 0.99990
2834e5e7: change from 0.99844 to 0.99990
ebc4685b: change from 0.97754 to 0.99990
156bd669: change from 0.99971 to 0.99990
8ccec255: change from 0.99997 to 0.99990
23d82bac: change from 0.88435 to 0.99990
9ac4b71e: change from 0.98820 to 0.99990
e6515685: change from 0.99940 to 0.99990
98cf7b29: change from 0.56669 to 0.99990
37cfb516: change from 0.98825 to 0.99990
4af76897: change from 0.99983 to 0.99990
8046c7d3: change from 0.99943 to 0.99990
707505e3: change from 0.00000 to 0.00010
cddbbe52: change from 0.99836 to 0.99990
00abc1a2: change from 0.99787 to 0.99990
a7f9dd0c: change from 0.99959 to 0.99990
301e2089: change from 0.99621 to 0.99990
76f51088: change from 0.99770 to 0.99990
bbef7d99: change from 0.99789 to 0.99990
87e7af15: change

In [12]:
concat_sub[['id', 'is_iceberg']].to_csv('blend_contrast_minimax_median_leakage_corrected_th{}.csv'.format(th), 
                                        index=False, float_format='%.15f')

# Generate Confident Pseudo Labels to train

In [ ]:
confident_mask = np.where(np.all(concat_sub.iloc[:,1:file_num+1] > cutoff_lo, axis=1), 
                                    True, # all > lo
                                    np.where(np.all(concat_sub.iloc[:,1:file_num+1] < cutoff_hi, axis=1),
                                             True, # all < hi 
                                             False # use the best
                                            )
                                   )
print(confident_mask)

[False False False ...,  True  True False]


In [ ]:
test_df = pd.read_json('../Data/no_denoise_processed_test.json')
test_df.sort_index(inplace=True)
print(test_df.head(3))
print(test_df[confident_mask].shape)

In [ ]:
test = test_df[confident_mask].reset_index(drop=True)

In [ ]:
test.is_iceberg = concat_sub[confident_mask].reset_index(drop=True).is_iceberg
print(test.shape)
print(test.head(3))

In [ ]:
test.to_json('../Data/public_lb_1229_cpl_test.json')